In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.arange(-1.0, 1.1, 0.1)
Y = np.arange(-1.0, 1.1, 0.1)

In [3]:
input_data = []
correct_data = []
for x in X:
    for y in Y:
        input_data.append([x, y])
        if y < np.sin(np.pi * x):
            correct_data.append([0, 1])
        else:
            correct_data.append([1, 0])

In [4]:
n_data = len(correct_data)

# numpy array 로 변환
input_data = np.array(input_data)
correct_data = np.array(correct_data)

In [5]:
n_in = 2
n_mid = 6
n_out = 2

In [6]:
wb_width = 0.01
eta = 0.1
epoch = 101
interval = 10

In [7]:
# 은닉층 클래스 설정
class MiddleLayer:
    def __init__(self, n_upper, n):
        self.w = wb_width * np.random.randn(n_upper, n)
        self.b = wb_width * np.random.randn(n)
        
    # 순전파 forward
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b 
        self.y = 1 / (1 + np.exp(-u)) # sigmoid 함수
        
    # 역전파 backward
    def backward(self, grad_y):
        delta = grad_y * (1 - self.y) * self.y # sigmoid 함수 미분
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)
        
    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

In [9]:
# 출력층 클래스
class OutputLayer:
    def __init__(self, n_upper, n):
        self.w = wb_width * np.random.randn(n_upper, n)
        self.b = wb_width * np.random.randn(n)
        
    # 순전파 forward
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b 
        self.y = np.exp(u) / np.sum(np.exp(u), axis=1, keepdims=True) # 소프트맥스 함수
        
    # 역전파 backward
    def backward(self, t):
        delta = self.y - t
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)
        
    def update(self, eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

In [10]:
middle_layer = MiddleLayer(n_in, n_mid)
output_layer = OutputLayer(n_mid, n_out)

In [1]:
#학습
sin_data = np.sin(np.pi * X)
for i in range(epoch):
    index_random = np.arange(n_data)
    np.random.shuffle(index_random)
    
    total_error = 0
    x_1 = []
    y_1 = []
    x_2 = []
    y_2 = []

    for idx in index_random:
        x = input_data[idx] # 입력
        t = correct_data[idx] # 정답
        
# 순전파
        middle_layer.forward(x.reshape(1, 2)) # 입력을 행렬로 변환
        output_layer.forward(middle_layer.y) 
        
# 역전파
        output_layer.backward(t.reshape(1, 2)) # 정답을 행렬로 변환
        middle_layer.backward(output_layer.grad_x)

# 가중치와 편향 수정
        middle_layer.update(eta)
        output_layer.update(eta)
        
        if i % interval == 0:
            y = output_layer.y.reshape(-1) # 행렬을 벡터로 되돌림
# 오차계산
            total_error += - np.sum(t * np.log(y + 1e-7))
# 출력 기록
            if y[0] > y[1]:
                x_1.append(x[0])
                y_1.append(x[1])
            else:
                x_2.append(x[0])
                y_2.append(x[1])
        
    if i % interval == 0:   
    # 출력 그래프 표시
            plt.plot(X, sin_data, linestyle="dashed")
            plt.scatter(x_1, y_1, marker="+")
            plt.scatter(x_2, y_2, marker="x")
            plt.show()

    # 에포크 수와 오차 표시
            print("Epoch:" + str(i) + "/" + str(epoch), "Error:" + str(total_error/n_data))

NameError: name 'np' is not defined